In [ ]:
# import required libraries
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics,
                        Metrics, component)
import os
import re
from pathlib import Path

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.cloud import storage
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op

import kfp
from kfp import dsl
# from kfp.v2.dsl import (Model, Input, component)
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,HTML,
                        OutputPath, ClassificationMetrics, Metrics, component)
from typing import NamedTuple
from google.cloud import bigquery
import logging 
from datetime import datetime

#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
DATASET_ID = 'call_to_retention_dataset'
RESOURCE_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
MODEL_ID = '5090'

project_id = PROJECT_ID
dataset_id = DATASET_ID 
resource_bucket = RESOURCE_BUCKET
bucket_name = BUCKET_NAME 

def get_gcp_bqclient(project_id, use_local_credential=True):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

client = get_gcp_bqclient(project_id)

job_config = bigquery.QueryJobConfig()

# Change dataset / table + sp table name to version in bi-layer
query =\
    f'''
        SELECT
            *
        FROM `bi-stg-divg-speech-pr-9d940b.tos_cross_sell_dataset.INFORMATION_SCHEMA.PARTITIONS`
        WHERE table_name='temp_tos_crosssell'

    '''

df = client.query(query, job_config=job_config).to_dataframe()

logging.info(df.to_string())

logging.info(f"Loaded {df.total_rows[0]} rows into \
         {df.table_catalog[0]}.{df.table_schema[0]}.{df.table_name[0]} on \
         {datetime.strftime((df.last_modified_time[0]), '%Y-%m-%d %H:%M:%S') } !")


query_rpc_flag_distribution = \
    f'''
        SELECT
            decile_grp_num,
            count(*) as cnt
        FROM `bi-stg-divg-speech-pr-9d940b.tos_cross_sell_dataset.temp_tos_crosssell` 
        GROUP BY decile_grp_num
    '''

rpc_flag_cnts = client.query(query_rpc_flag_distribution, job_config=job_config).to_dataframe()

# Log number of samples in each class
logging.info(f"RPC Flag 0: {rpc_flag_cnts.loc[rpc_flag_cnts.decile_grp_num==1].cnt.values[0]} samples!")
logging.info(f"RPC Flag 1: {rpc_flag_cnts.loc[rpc_flag_cnts.decile_grp_num==2].cnt.values[0]} samples!")

In [ ]:
print(logging.info)

In [ ]:
query =\
    f'''
       SELECT
            *
        FROM `bi-stg-divg-speech-pr-9d940b.tos_cross_sell_dataset.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
        WHERE table_name='temp_tos_crosssell'

    '''

df = client.query(query, job_config=job_config).to_dataframe()

col_list = list([ col for col in df.column_name if col != "rpc_flag"])    
print(col_list)

In [ ]:
nt = NamedTuple("output", [("col_list", list)])

In [ ]:
print(nt)